In [8]:
from sentence_transformers import SentenceTransformer
import os
import json
import pickle
import torch
import os
import json
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
root='/media/jh/新加卷1'
with open(f'{root}/2024_11_10/llms4subjects-main/shared-task-datasets/GND/dataset/GND-Subjects-all.json') as f:
    gnd_jsons=json.load(f)
labels=[]
for c in gnd_jsons:
    item1={}
    item1['sentence2']='Classification Name is '+c['Classification Name']+'. Name is '+c['Name']
    item1['Code']=c['Code']
    labels.append(item1)
from sentence_transformers import SentenceTransformer
import torch

# model_id='/home/jh/semeval/sentence-transformers/paraphrase-multilingual-mpnet-base-v2/'
# model = SentenceTransformer(model_id, model_kwargs={"torch_dtype": torch.float16, "device": "cuda"}) 
model_name_or_path='out_models/all-distilroberta-v1'#'/media/jh/新加卷/2025_1_16/sentence-transformers/all-mpnet-base-v2/' #models/gte_gnd 可以 
model = SentenceTransformer(model_name_or_path, model_kwargs={"torch_dtype": torch.float16}, trust_remote_code=True)#, 
# model=model.to('cuda')
# 把labels的一维张量转换为二维张量
batch_size=32
texts=[label['sentence2'] for label in labels]
texts=[texts[i:i+batch_size] for i in range(0, len(texts), batch_size)]
if len(texts)*batch_size<len(labels):
    texts.append([label['sentence2'] for label in labels[len(texts)*batch_size:]])
y_test=[]
for text in texts:
    with torch.no_grad():
        embeddings = model.encode(text, convert_to_tensor=True)
    item=embeddings.cpu()
    y_test.append(item)
print(embeddings)

y_t=[]
i=0
for y in y_test:
    for e in y:
        item={}
        item['Code']=labels[i]['Code']
        item['embedding']=e
        item['sentence2']=labels[i]['sentence2']
        y_t.append(item)
        i+=1
print(len(y_t),len(labels)) # 

def process_data(en_de,root,des_root,mode='m'):
    def get_train_data(json_file):
        with open(json_file,'r') as f:
            data=json.loads(f.read())
        for d in data['@graph']:
            if 'title' in d:
                tmp=d
                break
        item={}
        if type(tmp['title'])==list:
            item['title']=' '.join(tmp['title'])
        else:
            item['title']=tmp['title']
        if type(tmp['abstract'])==list:
            item['abstract']='  '.join(tmp['abstract'])
        else:
            item['abstract']=tmp['abstract']
        return item
    x_test=[]
    file_names=[]
    des_path=[]
    for dirs in os.listdir(f'{root}/'):
        if not os.path.exists(f'{root}/{dirs}/{en_de}'):
            continue
        for file in os.listdir(f'{root}/{dirs}/{en_de}'):
            # if os.path.exists(f'{des_root}/{dirs}/{en_de}/{file}'):
            #     continue
            item=get_train_data(f'{root}/{dirs}/{en_de}/{file}')
            text=item['title']+'.'+item['abstract']
            x_test.append(text)
            # 把file的后缀名去掉
            file=file.split('.')[0]+'.json'
            file_names.append(f'{des_root}/{dirs}/{en_de}/{file}')
            des_path.append(f'{des_root}/{dirs}/{en_de}')
    batch_size=768
    texts=[x_test[i:i+batch_size] for i in range(0, len(x_test), batch_size)]
    if len(texts)*batch_size<len(file_names):
        texts.append([file_names[len(texts)*batch_size:]])
    result=[]
    index=0
    index1=0
    
    for text in texts:
        with torch.no_grad():
            embeddings = model.encode(text, convert_to_tensor=True)
        y_list=[y['embedding'].tolist() for y in y_t]
        # y_tt=torch.tensor([y['embedding'].tolist() for y in y_t]).to('cuda')
        # print(embeddings.shape,y_tt.shape)
        similarities=[]
        embeddings=embeddings.to('cuda')
        for cc in range(0,len(y_list),batch_size):
            y_tt=torch.tensor(y_list[cc:cc+batch_size]).to('cuda')
            with torch.no_grad():
                similaritie = model.similarity(y_tt.to(torch.float16),embeddings.to(torch.float16)).to('cpu')
            # print(similaritie.shape)
            similarities.extend(similaritie.tolist())
            # print(torch.tensor(similarities).shape) #(n,768)
        if mode=='m2':
            m2_similarities.extend(torch.tensor(similarities).T.tolist())
            print(torch.tensor(m2_similarities).shape)
            continue
        elif mode=='m1':
            with torch.no_grad():
                similarities=torch.tensor(similarities).to('cuda')
                # print(similarities.shape) 上次保存的前500个
                top_k = torch.topk(similarities, k=500, dim=0).indices.T.cpu().tolist() # 取出相似度最高的50个
            for t in top_k:
                code=[]
                for i in t:
                    tmp={}
                    tmp['Code']=y_t[i]['Code']
                    tmp['Name']=y_t[i]['sentence2']
                    code.append(tmp)
                item1={}
                item1['subjects']=code
                item1['text']=x_test[index1]
                index1+=1
                g_result.append(item1)

        else:
            with torch.no_grad():
                similarities=torch.tensor(similarities).to('cuda')
                # print(similarities.shape) 上次保存的前500个
                top_k = torch.topk(similarities, k=50, dim=0).indices.T.cpu().tolist() # 取出相似度最高的50个
            for t in top_k:
                code=[]
                for i in t:
                    code.append(y_t[i]['Code'])
                item1={}
                item1['dcterms:subject']=code
                result.append(item1)
            for res in result:
                # 如果不存在目录，则创建目录
                if not os.path.exists(des_path[index]):
                    os.makedirs(des_path[index])
                # if os.path.exists(file_names[index]):
                #     index+=1
                #     continue
                with open(file_names[index],'w') as f:
                    json.dump(res,f,ensure_ascii=False,indent=4)
                index+=1
            print('batch',index)
            result=[] #重置result
# 打印运行时间
import time
start_time = time.time()
root1='/media/jh/新加卷1'
root='/media/jh/新加卷1/2024_11_10/llms4subjects-main/shared-task-datasets/TIBKAT/all-subjects/data/dev/'
des_root='/media/jh/新加卷1/2024_12_04/2025_1/test1'
# model_id='./models/mpnet_gnd_triplet'#'/media/jh/新加卷/2025_1_16/final_model'
# # with open('./tmp/core_labels_gte.pkl','rb') as f:
# #     y_t=pickle.load(f)
# model = SentenceTransformer(model_id, model_kwargs={"torch_dtype": torch.float16, "device": "cuda"}) 

# m2_similarities=[]
g_result=[] #存放预测的top500
process_data('en',root,des_root,mode='m1') #15904
# with open('./tmp/m2_en.pkl', 'wb') as f:
#     pickle.dump(m2_similarities, f)
# m2_similarities=[]
# process_data('de',root,des_root,mode='m1') #15904
# with open('./tmp/m2_de.pkl', 'wb') as f:
#     pickle.dump(m2_similarities, f)
end_time = time.time()
print('运行时间:', end_time - start_time, '秒')

tensor([[ 0.0254,  0.0038,  0.0007,  ...,  0.0342,  0.0229, -0.0006],
        [ 0.0014,  0.0028, -0.0237,  ...,  0.0449,  0.0090, -0.0242],
        [-0.0166,  0.0191,  0.0038,  ...,  0.0351,  0.0049, -0.0182]],
       device='cuda:0', dtype=torch.float16)
204739 204739
运行时间: 385.65167784690857 秒


In [14]:
# !pip install openai

from openai import OpenAI

client = OpenAI(
    base_url='https://api-inference.modelscope.cn/v1/',
    api_key='XXXX', # ModelScope Token
)
def llm_process(prompt, userinput):
    response = client.chat.completions.create(
        model='deepseek-ai/DeepSeek-R1-Distill-Llama-70B', # ModelScope Model-Id
        messages=[
            {
                'role': 'system',
                'content': prompt
            },
            {
                'role': 'user',
                'content': userinput
            }
        ],
        stream=False
    )

    print(response.choices[0].message.reasoning_content)
    print('\n\n === Final Answer ===\n')
    print(response.choices[0].message.content)
    return response.choices[0].message.content

def process2(data):
    prompt='''请从labels中选择前50个与text最匹配的labels，以json格式输出它们的Code与Name(只取前50字符)'''
    userinput='''{\'test\':'''+data['text']+''', \'labels\':['''
    # 把data['subjects']中的每个元素转为字符串
    for d in data['subjects']:
        userinput+=str(d)+','
    userinput=userinput[:-1]+']}'
    return llm_process(prompt, userinput)
resList=[]
cnt=10
for data in g_result:
    resList.append(process2(data))
    cnt-=1
    if cnt==0:
        break
print(resList)



Okay, so I need to find the top 50 labels from the given list that best match the provided text. The text is about turbulent pipe flow, very-large-scale motions, and their effects on one- and two-point statistics. It also mentions direct numerical simulations and the interaction between these large-scale motions and near-wall turbulence. The labels provided are a list of various mechanical, thermal, and acoustic classifications, each with a code and a name.

First, I should understand the main topics of the text. The key terms here are turbulent pipe flow, very-large-scale motions, Reynolds number, direct numerical simulations (DNS), one-point and two-point statistics, conditional averaging, and the interaction between high-speed and low-speed structures. 

Next, I need to go through each label and see how well they relate to the text. The labels are in German, so I need to translate them or understand their meaning based on the structure given. For example, "gnd:4117265-6" is about t

InternalServerError: <!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html>
<head><title>504 Gateway Time-out</title></head>
<body>
<center><h1>504 Gateway Time-out</h1></center>
<hr/>Powered by Tengine<hr><center>nginx</center>
</body>
</html>